In [1]:
import dataset  # Assuming the provided code is saved in a file named 'dataset.py'
from pathlib import Path

# Define the path to your data and the split you want to use (e.g., 'train')
data_path = Path("/Users/junyulyu/Desktop/Capstone/prepare_dataset")
split = "train"  # or 'val' or 'test', depending on your dataset_splits.csv

# Instantiate the dataset
forest_sat_dataset = dataset.MineSATDataset(split=split, data_path=data_path)

# Choose an index of an image you want to visualize
index = 3  # You can change this to any valid index

# Display the transformed images including NBR and NDVI
forest_sat_dataset.display_transformed_images(index)


/Users/junyulyu/Library/Caches/pypoetry/virtualenvs/mine-seg-sat-14bnJD6w-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: '/Users/junyulyu/Desktop/Capstone/prepare_dataset/mask_directory6/mask.tif'